In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import re
import numpy as np
pd.options.display.max_columns = None
pd.set_option('display.max_rows', None)

In [9]:
# Load the CSV into the DataFrame
staphylococcaceae_df = pd.read_csv(r"D:\Vivli 2024\Jupiter Notebooks\Vidata Analysis\Filtered Data for the Species and Families\staphylococcaceae_df.csv")

# Display the DataFrame (optional)
staphylococcaceae_df.head()


C:\Users\Jude\AppData\Local\Temp\ipykernel_2436\535289267.py:2: DtypeWarning:

Columns (9,13,15) have mixed types. Specify dtype option on import or set low_memory=False.



,Isolate Number,Data Source,Species,Family,Gender,Age Group,Country,Year,Source of Infection,Amoxicillin-clavulanate,Amoxicillin-clavulanate_I,Ampicillin,Ampicillin_I,Piperacillin tazobactam,Piperacillin tazobactam_I,Ceftriaxone,Ceftriaxone_I,Ceftazidime,Ceftazidime_I,Cefepime,Cefepime_I,Imipenem,Imipenem_I,Meropenem,Meropenem_I,Ciprofloxacin,Ciprofloxacin_I,Levofloxacin,Levofloxacin_I,Amikacin,Amikacin_I,Gentamicin,Gentamicin_I,Vancomycin,Vancomycin_I,Azithromycin,Azithromycin_I,Clarithromycin,Clarithromycin_I,Tigecycline,Tigecycline_I,Linezolid,Linezolid_I,Trimethoprim-sulfamethoxazole,Trimethoprim-sulfamethoxazole_I
0,1773252,ATLAS,Staphylococcus epidermidis,Staphylococcaceae,Male,65 to 84 Years,Spain,2018,Abscess,NaN,NaN,>8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,>4,NaN,NaN,NaN,>16,NaN,1,NaN,NaN,NaN,NaN,NaN,0.25,NaN,1,NaN,>2,NaN
1,1773253,ATLAS,Staphylococcus epidermidis,Staphylococcaceae,Female,65 to 84 Years,Spain,2018,Wound,NaN,NaN,<=0.25,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.25,NaN,NaN,NaN,<=1,NaN,2,NaN,NaN,NaN,NaN,NaN,0.12,NaN,1,NaN,0.25,NaN
2,1773254,ATLAS,Staphylococcus epidermidis,Staphylococcaceae,Male,19 to 64 Years,Spain,2018,Peritoneal Fluid,NaN,NaN,>8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,>4,NaN,NaN,NaN,>16,NaN,2,NaN,NaN,NaN,NaN,NaN,0.12,NaN,1,NaN,>2,NaN
3,1773255,ATLAS,Staphylococcus epidermidis,Staphylococcaceae,Female,19 to 64 Years,Spain,2018,Abscess,NaN,NaN,>8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,>4,NaN,NaN,NaN,>16,NaN,2,NaN,NaN,NaN,NaN,NaN,0.12,NaN,>8,NaN,>2,NaN
4,1773256,ATLAS,Staphylococcus epidermidis,Staphylococcaceae,Female,65 to 84 Years,Spain,2018,Abscess,NaN,NaN,>8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,>4,NaN,NaN,NaN,>16,NaN,2,NaN,NaN,NaN,NaN,NaN,0.12,NaN,>8,NaN,>2,NaN


In [10]:
# Function to extract numeric value from a string, handling cases like '<', '>', etc.
def extract_numeric_value(value):
    if isinstance(value, str):
        # Use regex to remove any non-numeric characters except for '.' and '-'
        cleaned_value = re.sub(r'[^\d.-]', '', value)
        try:
            return float(cleaned_value)
        except ValueError:
            return np.nan  # Return NaN if conversion fails
    return value  # Return the value as is if it's already numeric

In [11]:
# Function to interpret antibiotic values based on the given conditions for Staphylococcaceae family
def interpret_antibiotic_staphylococcaceae(row, antibiotic, year, species):
    value = extract_numeric_value(row[antibiotic])  # Clean the value

    # Handle missing or non-numeric values
    if pd.isna(value):
        return 'Use Not recommended'

    # Interpret for Penicillins (AMC, AMP, TZP)
    if antibiotic == 'Amoxicillin-clavulanate':
        return 'Susceptible' if value <= 8 else 'Resistant'
    elif antibiotic == 'Ampicillin':
        return 'Susceptible' if value <= 2 else 'Resistant'
    elif antibiotic == 'Piperacillin tazobactam':
        return 'Susceptible' if value <= 16 else 'Resistant'

    # Interpret for Cephalosporins (CAZ, AXO, PIP)
    elif antibiotic == 'Ceftazidime':
        return 'Not Applicable'
    elif antibiotic == 'Ceftriaxone':
        return 'Susceptible' if value <= 4 else 'Resistant'
    elif antibiotic == 'Cefepime':
        if value <= 2:
            return 'Susceptible'
        elif 2.1 <= value <= 4:
            return 'Intermediate'
        else:
            return 'Resistant'

    # Interpret for Carbapenems (IPM, MEM)
    elif antibiotic == 'Imipenem':
        return 'Susceptible' if value <= 4 else 'Resistant'
    elif antibiotic == 'Meropenem':
        return 'Susceptible' if value <= 2 else 'Resistant'

    # Interpret for Fluoroquinolones (CIP, LEV)
    elif antibiotic in ['Ciprofloxacin', 'Levofloxacin']:
        if year in [2022, 2021, 2020]:
            if value <= 0.001:
                return 'Susceptible'
            elif 0.002 <= value <= 1:
                return 'Intermediate'
            else:
                return 'Resistant'
        elif year in [2019, 2018]:
            return 'Susceptible' if value <= 1 else 'Resistant'

    # Interpret for Aminoglycosides (AMK, GEN)
    elif antibiotic == 'Amikacin':
        if year == 2022:
            return 'Susceptible' if value <= 16 else 'Resistant'
        elif year in [2021, 2020]:
            return 'Susceptible' if value <= 8 else 'Resistant'
        elif year in [2019, 2018]:
            if value <= 8:
                return 'Susceptible'
            elif 8.1 <= value <= 16:
                return 'Intermediate'
            else:
                return 'Resistant'
    elif antibiotic == 'Gentamicin':
        if year == 2022:
            return 'Susceptible' if value <= 2 else 'Resistant'
        else:
            return 'Susceptible' if value <= 1 else 'Resistant'

    # Interpret for Glycopeptide (VAN)
    elif antibiotic == 'Vancomycin':
        if species == 'Staphylococcus aureus':
            return 'Susceptible' if value <= 2 else 'Resistant'
        else:
            return 'Susceptible' if value <= 4 else 'Resistant'

    # Interpret for Macrolides (AZM, CAM)
    elif antibiotic == 'Azithromycin':
        if year == 2022:
            return 'Susceptible' if value <= 2 else 'Resistant'
        else:
            if value <= 1:
                return 'Susceptible'
            elif 1.1 <= value <= 2:
                return 'Intermediate'
            else:
                return 'Resistant'
    elif antibiotic == 'Clarithromycin':
        if value <= 1:
            return 'Susceptible'
        elif 1.1 <= value <= 2:
            return 'Intermediate'
        else:
            return 'Resistant'

    # Interpret for Tetracyclines (TIG)
    elif antibiotic == 'Tigecycline':
        return 'Susceptible' if value <= 0.5 else 'Resistant'

    # Interpret for Oxazolidines (LZD)
    elif antibiotic == 'Linezolid':
        return 'Susceptible' if value <= 4 else 'Resistant'

    # Interpret for Antifolate (SXT)
    elif antibiotic == 'Trimethoprim-sulfamethoxazole':
        if value <= 2:
            return 'Susceptible'
        elif 2.1 <= value <= 4:
            return 'Intermediate'
        else:
            return 'Resistant'

    return 'Use Not recommended'

# Apply the function to the relevant columns for Staphylococcaceae family
def apply_interpretation_staphylococcaceae(row):
    antibiotics = [
        'Amoxicillin-clavulanate', 'Ampicillin', 'Piperacillin tazobactam', 'Ceftazidime', 'Ceftriaxone',
        'Cefepime', 'Imipenem', 'Meropenem', 'Ciprofloxacin', 'Levofloxacin', 'Amikacin', 'Gentamicin',
        'Vancomycin', 'Azithromycin', 'Clarithromycin', 'Tigecycline', 'Linezolid', 'Trimethoprim-sulfamethoxazole'
    ]
    year = row['Year']
    species = row['Species']

    for antibiotic in antibiotics:
        interpretation = interpret_antibiotic_staphylococcaceae(row, antibiotic, year, species)
        row[antibiotic + '_I'] = interpretation  # Fill the interpretation into the respective "_I" column
    return row

# Apply the interpretation across the dataframe for Staphylococcaceae family
staphylococcaceae_df = staphylococcaceae_df.apply(apply_interpretation_staphylococcaceae, axis=1)

# Display the result
staphylococcaceae_df.head()


,Isolate Number,Data Source,Species,Family,Gender,Age Group,Country,Year,Source of Infection,Amoxicillin-clavulanate,Amoxicillin-clavulanate_I,Ampicillin,Ampicillin_I,Piperacillin tazobactam,Piperacillin tazobactam_I,Ceftriaxone,Ceftriaxone_I,Ceftazidime,Ceftazidime_I,Cefepime,Cefepime_I,Imipenem,Imipenem_I,Meropenem,Meropenem_I,Ciprofloxacin,Ciprofloxacin_I,Levofloxacin,Levofloxacin_I,Amikacin,Amikacin_I,Gentamicin,Gentamicin_I,Vancomycin,Vancomycin_I,Azithromycin,Azithromycin_I,Clarithromycin,Clarithromycin_I,Tigecycline,Tigecycline_I,Linezolid,Linezolid_I,Trimethoprim-sulfamethoxazole,Trimethoprim-sulfamethoxazole_I
0,1773252,ATLAS,Staphylococcus epidermidis,Staphylococcaceae,Male,65 to 84 Years,Spain,2018,Abscess,NaN,Use Not recommended,>8,Resistant,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,>4,Resistant,NaN,Use Not recommended,>16,Resistant,1,Susceptible,NaN,Use Not recommended,NaN,Use Not recommended,0.25,Susceptible,1,Susceptible,>2,Susceptible
1,1773253,ATLAS,Staphylococcus epidermidis,Staphylococcaceae,Female,65 to 84 Years,Spain,2018,Wound,NaN,Use Not recommended,<=0.25,Susceptible,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,0.25,Susceptible,NaN,Use Not recommended,<=1,Susceptible,2,Susceptible,NaN,Use Not recommended,NaN,Use Not recommended,0.12,Susceptible,1,Susceptible,0.25,Susceptible
2,1773254,ATLAS,Staphylococcus epidermidis,Staphylococcaceae,Male,19 to 64 Years,Spain,2018,Peritoneal Fluid,NaN,Use Not recommended,>8,Resistant,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,>4,Resistant,NaN,Use Not recommended,>16,Resistant,2,Susceptible,NaN,Use Not recommended,NaN,Use Not recommended,0.12,Susceptible,1,Susceptible,>2,Susceptible
3,1773255,ATLAS,Staphylococcus epidermidis,Staphylococcaceae,Female,19 to 64 Years,Spain,2018,Abscess,NaN,Use Not recommended,>8,Resistant,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,>4,Resistant,NaN,Use Not recommended,>16,Resistant,2,Susceptible,NaN,Use Not recommended,NaN,Use Not recommended,0.12,Susceptible,>8,Resistant,>2,Susceptible
4,1773256,ATLAS,Staphylococcus epidermidis,Staphylococcaceae,Female,65 to 84 Years,Spain,2018,Abscess,NaN,Use Not recommended,>8,Resistant,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,NaN,Use Not recommended,>4,Resistant,NaN,Use Not recommended,>16,Resistant,2,Susceptible,NaN,Use Not recommended,NaN,Use Not recommended,0.12,Susceptible,>8,Resistant,>2,Susceptible


In [12]:
# Dictionary for Staphylococcaceae species
staphylococcaceae_species_list = [
    'Staphylococcus aureus', 'Staphylococcus capitis', 'Staphylococcus cohnii', 
    'Staphylococcus epidermidis', 'Staphylococcus hominis', 'Staphylococcus hyicus', 
    'Staphylococcus lugdunensis', 'Staphylococcus pettenkoferi', 'Staphylococcus saprophyticus', 
    'Staphylococcus schleiferi', 'Staphylococcus sciuri', 'Staphylococcus simulans', 
    'Staphylococcus warneri', 'Staphylococcus xylosus'
]

# List of antibiotics of interest with interpretations (_I)
antibiotics_list = [
    'Amoxicillin-clavulanate_I', 'Ampicillin_I', 'Piperacillin tazobactam_I', 
    'Ceftriaxone_I', 'Ceftazidime_I', 'Cefepime_I', 'Imipenem_I', 'Meropenem_I', 
    'Ciprofloxacin_I', 'Levofloxacin_I', 'Amikacin_I', 'Gentamicin_I', 
    'Vancomycin_I', 'Azithromycin_I', 'Clarithromycin_I', 'Tigecycline_I', 
    'Linezolid_I', 'Trimethoprim-sulfamethoxazole_I'
]

# List of years of interest
years_list = [2018, 2019, 2020, 2021, 2022]

# Dictionary to store susceptibility results for Staphylococcaceae
staphylococcaceae_results = {}

# Loop through each species of interest in Staphylococcaceae
for species in staphylococcaceae_species_list:
    # Filter the data for the current species
    df_species = staphylococcaceae_df[staphylococcaceae_df['Species'] == species]

    # Dictionary to store antibiotic results
    antibiotic_results = {}

    # Loop through each antibiotic of interest
    for antibiotic in antibiotics_list:
        susceptibility_rates_list = []
        
        # Loop through each year
        for year in years_list:
            year_data = df_species[df_species['Year'] == year]
            total_count = len(year_data)
            
            if total_count > 0:
                # Calculate susceptibility counts
                susceptible_count = (year_data[antibiotic] == 'Susceptible').sum()
                intermediate_count = (year_data[antibiotic] == 'Intermediate').sum()
                resistant_count = (year_data[antibiotic] == 'Resistant').sum()

                # Calculate rates
                total_reported = susceptible_count + intermediate_count + resistant_count
                if total_reported > 0:
                    susceptible_rate = (susceptible_count / total_reported) * 100
                    intermediate_rate = (intermediate_count / total_reported) * 100
                    resistant_rate = (resistant_count / total_reported) * 100

                    # Round rates to 2 decimal places
                    susceptible_rate = round(susceptible_rate, 2)
                    intermediate_rate = round(intermediate_rate, 2)
                    resistant_rate = round(resistant_rate, 2)

                    # Append the data to the list
                    susceptibility_rates_list.append({
                        'Year': year,
                        'Susceptible': susceptible_rate,
                        'Intermediate': intermediate_rate,
                        'Resistant': resistant_rate
                    })
        
        # Store the results for this antibiotic
        if susceptibility_rates_list:
            antibiotic_results[antibiotic] = pd.DataFrame(susceptibility_rates_list)

    # Store the results for this species
    if antibiotic_results:
        staphylococcaceae_results[species] = antibiotic_results

# Final results: Display susceptibility rates for Staphylococcaceae species grouped by years
for species, antibiotics_data in staphylococcaceae_results.items():
    print(f"Results for Species: {species}")
    for antibiotic, df in antibiotics_data.items():
        print(f"\nSusceptibility rates for {antibiotic}:")
        print(df)
        print("\n" + "="*50 + "\n")


Results for Species: Staphylococcus aureus

Susceptibility rates for Amoxicillin-clavulanate_I:
   Year  Susceptible  Intermediate  Resistant
0  2019        100.0           0.0        0.0



Susceptibility rates for Ampicillin_I:
   Year  Susceptible  Intermediate  Resistant
0  2018        39.51           0.0      60.49
1  2019        38.70           0.0      61.30
2  2020        41.44           0.0      58.56
3  2021        46.57           0.0      53.43
4  2022        48.85           0.0      51.15



Susceptibility rates for Piperacillin tazobactam_I:
   Year  Susceptible  Intermediate  Resistant
0  2019        100.0           0.0        0.0



Susceptibility rates for Ceftriaxone_I:
   Year  Susceptible  Intermediate  Resistant
0  2019        100.0           0.0        0.0



Susceptibility rates for Meropenem_I:
   Year  Susceptible  Intermediate  Resistant
0  2019        100.0           0.0        0.0



Susceptibility rates for Levofloxacin_I:
   Year  Susceptible  Intermediate 

In [13]:
import plotly.express as px
import pandas as pd

# Dictionary grouping antibiotics by their pharmacological classes
antibiotic_groups = {
    'Penicillins': ['Amoxicillin-clavulanate_I', 'Ampicillin_I', 'Piperacillin tazobactam_I'],
    'Cephalosporins': ['Ceftriaxone_I', 'Ceftazidime_I', 'Cefepime_I'],
    'Carbapenems': ['Imipenem_I', 'Meropenem_I'],
    'Fluoroquinolones': ['Ciprofloxacin_I', 'Levofloxacin_I'],
    'Aminoglycosides': ['Amikacin_I', 'Gentamicin_I'],
    'Glycopeptides': ['Vancomycin_I'],
    'Macrolides': ['Azithromycin_I', 'Clarithromycin_I'],
    'Tetracyclines': ['Tigecycline_I'],
    'Oxazolidinones': ['Linezolid_I'],
    'Antifolates': ['Trimethoprim-sulfamethoxazole_I']
}

# Species list for Staphylococcaceae family
staphylococcaceae_species_list = [
    'Staphylococcus aureus', 'Staphylococcus capitis', 'Staphylococcus cohnii', 
    'Staphylococcus epidermidis', 'Staphylococcus hominis', 'Staphylococcus hyicus', 
    'Staphylococcus lugdunensis', 'Staphylococcus pettenkoferi', 'Staphylococcus saprophyticus', 
    'Staphylococcus schleiferi', 'Staphylococcus sciuri', 'Staphylococcus simulans', 
    'Staphylococcus warneri', 'Staphylococcus xylosus'
]

# Loop through each species in Staphylococcaceae family
for species, antibiotics_data in staphylococcaceae_results.items():
    if species in staphylococcaceae_species_list:
        print(f"Visualizing results for Species: {species}")

        # Loop through each antibiotic group
        for group, antibiotics in antibiotic_groups.items():
            print(f"Visualizing results for Antibiotic Group: {group}")

            # Prepare a combined dataframe for the current group
            combined_df_list = []

            for antibiotic in antibiotics:
                if antibiotic in antibiotics_data:
                    # Add the 'Antibiotic' column to identify each antibiotic in the group
                    df = antibiotics_data[antibiotic].copy()
                    df['Antibiotic'] = antibiotic.replace('_I', '')  # Removing the "_I" suffix for readability
                    combined_df_list.append(df)

            # If there is data for this group, combine and plot
            if combined_df_list:
                combined_df = pd.concat(combined_df_list)

                # Plotting trends for each antibiotic in the group
                fig = px.line(
                    combined_df,
                    x='Year',
                    y=['Susceptible', 'Intermediate', 'Resistant'],
                    color='Antibiotic',  # Different colors for each antibiotic
                    line_dash='variable',  # Different dash patterns for susceptible, intermediate, and resistant
                    title=f"Susceptibility Trends for {species} ({group} Group)",
                    labels={'value': 'Percentage (%)', 'variable': 'Resistance Category'},
                    markers=True
                )

                # Customize the layout
                fig.update_layout(
                    xaxis_title="Year",
                    yaxis_title="Percentage (%)",
                    legend_title="Antibiotic / Resistance Category",
                    template="plotly",
                    width=1000,  # Adjust the width
                    height=600,  # Adjust the height
                    xaxis=dict(tickmode='linear', tick0=2018, dtick=1)  # Ensure integer ticks for years
                )

                # Show the plot
                fig.show()


Visualizing results for Species: Staphylococcus aureus
Visualizing results for Antibiotic Group: Penicillins


Visualizing results for Antibiotic Group: Cephalosporins


Visualizing results for Antibiotic Group: Carbapenems


Visualizing results for Antibiotic Group: Fluoroquinolones


Visualizing results for Antibiotic Group: Aminoglycosides


...

Visualizing results for Antibiotic Group: Aminoglycosides


Visualizing results for Antibiotic Group: Glycopeptides


Visualizing results for Antibiotic Group: Macrolides
Visualizing results for Antibiotic Group: Tetracyclines


Visualizing results for Antibiotic Group: Oxazolidinones


Visualizing results for Antibiotic Group: Antifolates


In [14]:
def check_increasing_trend(resistant_rates): 
    """
    This function checks if the resistance trend is increasing year over year.
    Returns True if resistance is increasing, False otherwise.
    """
    # Compare each element with the next one to ensure it's increasing
    for i in range(len(resistant_rates) - 1):
        if resistant_rates[i] > resistant_rates[i + 1]:
            return False  # If any rate decreases, return False
    return True  # Return True if all rates are increasing

def calculate_absolute_change(start_value, end_value):
    """
    This function calculates the absolute change between two values.
    """
    return end_value - start_value

# List to store results of increasing resistance trends
increasing_resistance_results = []

# Species list for Staphylococcaceae family
staphylococcaceae_species_list = [
    'Staphylococcus aureus', 'Staphylococcus capitis', 'Staphylococcus cohnii', 
    'Staphylococcus epidermidis', 'Staphylococcus hominis', 'Staphylococcus hyicus', 
    'Staphylococcus lugdunensis', 'Staphylococcus pettenkoferi', 'Staphylococcus saprophyticus', 
    'Staphylococcus schleiferi', 'Staphylococcus sciuri', 'Staphylococcus simulans', 
    'Staphylococcus warneri', 'Staphylococcus xylosus'
]

# Loop through each species in the results, specifically focusing on Staphylococcaceae species
for species, antibiotics_data in staphylococcaceae_results.items():
    if species in staphylococcaceae_species_list:
        # Loop through each antibiotic in the results
        for antibiotic, df in antibiotics_data.items():
            # Extract the resistant rates over the years
            resistant_rates = df['Resistant'].values
            years = df['Year'].values
            
            # Check if the resistance trend is increasing
            if check_increasing_trend(resistant_rates):
                # Calculate absolute changes between consecutive years
                absolute_changes = []
                for i in range(1, len(resistant_rates)):
                    start_value = resistant_rates[i - 1]
                    end_value = resistant_rates[i]
                    absolute_change = calculate_absolute_change(start_value, end_value)
                    absolute_changes.append(absolute_change)
                
                # Calculate the mean of the absolute changes
                if absolute_changes:
                    mean_absolute_change = sum(absolute_changes) / len(absolute_changes)
                else:
                    mean_absolute_change = 0.0
                
                # Store the information in the results
                increasing_resistance_results.append({
                    'Species': species,
                    'Antibiotic': antibiotic.replace('_I', ''),  # Remove "_I" for readability
                    'Years': ', '.join(map(str, years)),  # Concatenate years for display
                    'Resistant Rates': ', '.join(map(str, resistant_rates)),  # Concatenate rates for display
                    'Overall Resistance Rate Increase': round(mean_absolute_change, 2)  # Round to 2 decimal places
                })

# Convert the results into a DataFrame for easy display
increasing_resistance_df = pd.DataFrame(increasing_resistance_results)

# Display the results
if not increasing_resistance_df.empty:
    print("Species and Antibiotics showing increasing resistance trends with Overall Resistance Rate Increase:")
    print(increasing_resistance_df.to_string(index=False))  # Display DataFrame without the index
else:
    print("No increasing resistance trends identified.")


Species and Antibiotics showing increasing resistance trends with Overall Resistance Rate Increase:
                     Species                    Antibiotic                        Years          Resistant Rates  Overall Resistance Rate Increase
       Staphylococcus aureus       Amoxicillin-clavulanate                         2019                      0.0                              0.00
       Staphylococcus aureus       Piperacillin tazobactam                         2019                      0.0                              0.00
       Staphylococcus aureus                   Ceftriaxone                         2019                      0.0                              0.00
       Staphylococcus aureus                     Meropenem                         2019                      0.0                              0.00
       Staphylococcus aureus                  Azithromycin                         2019                      0.0                              0.00
      Staphylococc